 <div style="text-align:center;">
  <span style="color:green; font-size:2em; font-weight:bold;">Visualisation</span><br><br>

Dans cette partie nous allons nous interesser à la visualisation de nos données. Pour les taux de chomage et de croissance, nous allons se concentrer uniquement sur les observations de la période s'etallant de **2019** à **2023**. Nous voulons à partir de cela visualiser les niveaux moyens de chomage et de croissance économique pour les cinq dernières années.
L'objectif ici est de mettre en pratique les méthodes vues en cours.

# <span style="color:green">I. Taux de chômage à travers le monde</span><br><br> 

In [ ]:
import declarations as d 

In [ ]:
# Importation de la DF construite initialement dans le fichier "Préparation_données_copy.ipynb"
df_Unemployement= d.pd.read_csv('C:/Projet-Python/monmodule/bases/Unemployement_rates.csv', index_col=0)

In [ ]:
# Visualisation
df_Unemployement.head()

In [ ]:
# Informations sur la DF (nombre de valeurs non nulles, type de données de chaque colonne...)
df_Unemployement.info()

In [ ]:
# Convertir l'index en type date
df_Unemployement.index = d.pd.to_datetime(df_Unemployement.index)

In [ ]:
# Tronquer la DataFrame pour garder uniquement les lignes à partir de 2019
df_tronque = df_Unemployement.loc[df_Unemployement.index >= '2019']

# Afficher la DataFrame tronquée
df_tronque.head()

In [ ]:
# Calculer la moyenne des taux de chomage sur la période considérée
unemployement_mean = df_tronque.mean()

# Transformer en DF pandas
unemployement_mean = d.pd.DataFrame(unemployement_mean)

# Renommer la colonne 0
unemployement_mean.rename(columns={0: 'Unemployment mean'}, inplace=True)

# Renommer l'index
unemployement_mean.index.name = 'Country'

# Visualisation
unemployement_mean.head(50)

## <span style="color:green">I.1. Carte statique </span><br><br> 

## <span style="color:green">I.1. Carte interactive : cartes choroplèthes  </span><br><br>  

Une carte choroplèthe est une carte composée de polygones colorés. Il est utilisé pour représenter les variations spatiales d’une quantité.

In [ ]:
import plotly.express as px
import geopandas as gpd
# Créer une carte centrée sur la position moyenne des pays
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
merged = gpd.GeoDataFrame(d.pd.merge(world, unemployement_mean, left_on='name', right_on='Country', how='left'))
map_center = [merged['geometry'].centroid.y.mean(), merged['geometry'].centroid.x.mean()]

# Utiliser Plotly Express pour créer la carte choroplèthe
fig = px.choropleth(
    merged,
    geojson=merged.geometry,
    locations=merged.index,
    color='Unemployment mean',
    color_continuous_scale="Viridis",
    range_color=(merged['Unemployment mean'].min(), merged['Unemployment mean'].max()),
    labels={'Unemployment mean': 'Unemployment Rate (%)', 'name': 'Country'},
    title='Unemployment Rate by Country',
    hover_name='name'
)

# Ajouter une couche choroplèthe à la carte
fig.update_geos(fitbounds="locations", visible=True)

fig.update_layout ( margin = { "r" : 0 , "t" : 0 , "l" : 0 , "b" : 0 }) 

# Afficher la carte interactive
fig.show()

In [ ]:
world.head(20)
merged.head(20)

In [ ]:
#merged.set_index('name', inplace=True)
#merged.rename_axis('Pays', inplace=True)

In [ ]:
merged.head(5)
merged.shape

In [ ]:
import folium

# Créer une carte folium centrée sur la position moyenne des pays
map_center = [merged['geometry'].centroid.y.mean(), merged['geometry'].centroid.x.mean()]
mymap = folium.Map(location=map_center, zoom_start=2)

# Ajouter une couche choroplèthe avec les taux de chômage
folium.Choropleth(
    geo_data=merged,
    name='choropleth',
    data=merged,
    columns=["name", 'Unemployment mean'],
    key_on='feature.properties.iso_a3',
    fill_color="BuPu",
    fill_opacity=0.2,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(mymap)

# Ajouter des popups pour afficher les taux de chômage
for idx, row in merged.iterrows():
    # Add Marker only if Unemployment rate is not NaN
    if not d.pd.isna(row['Unemployment mean']):
        folium.Marker(
            location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
            popup=f"{row['name']}<br>Unemployment Rate: {row['Unemployment mean']}",
            icon=folium.Icon(icon='star', color='green')
        ).add_to(mymap)

# Afficher la carte interactive
mymap

In [119]:
# Importation de la DF construite initialement dans le fichier "Préparation_données_copy.ipynb"
df_Unemployement_L= d.pd.read_csv('C:/Projet-Python/monmodule/bases/final_data.csv', index_col=0)

In [122]:
import pandas as pd
import plotly.express as px

# Assuming your DataFrame is named df_long
# Make sure the index is already a datetime index
df_Unemployement_L.index = pd.to_datetime(df_Unemployement_L.index)

# Create an interactive time series plot with Plotly Express
fig = px.line(df_Unemployement_L, x=df_Unemployement_L.index, y='Unemployment_rate', color='COUNTRY',
              labels={'index': 'Date', 'Unemployment_rate': 'Unemployment Rate'},
              title='Série temporelle du taux de chômage par pays',
              template='plotly_dark')

# Show the interactive plot
fig.show()


# <span style="color:green">II. Croissance économique à travers le monde</span><br><br> 